# Logging Tracing Concept for Email Workflow

As the production system is not ready yet, and even if it would be there is not enough time to gather enough test data
to validate the concept necessary for demonstration.

To properly validate the concept the demo has to check the requirements below:
* simulate every component existing in future production system
* simulate errors / full failures in specific components to validate concept
* produce large amounts of realistic test data in a short amount of time

In [31]:
import sys
import random
import datetime

import logging
from logging import config

In [32]:
case_format = "CASE::%(case)s::%(levelname)s::%(message)s"
debug_format = "DEBUG::%(component)s::%(asctime)s::%(levelname)s::%(message)s"
debug_date_format = "%Y-%m-%d %H:%M:%S"

def get_handlers(name, level, console):
    handler_dict = {
        f'{name}_file': {
            'class': 'logging.FileHandler',
            'level': level,
            'formatter': 'debug' if name == 'debug' else 'case',
            'filename': f'./log/{name}.log'
        },
    }
    if console:
        handler_dict[f'{name}_console'] = {
            'class': 'logging.StreamHandler',
            'level': level,
            'formatter': 'debug' if name == 'debug' else 'case',
            'stream': 'ext://sys.stdout'
        }
    return handler_dict

def setup_logger(name, console=True):
    level = logging.DEBUG
    handlers = [ f'{name}_file' ]
    if console:
        handlers.append(f'{name}_console')
    config.dictConfig({
        'version': 1,
        'disable_existing_loggers': False,
        'formatters': {
            'debug': {'format': debug_format, 'datefmt': debug_date_format},
            'case': {'format': case_format}
        },
        'handlers': get_handlers(name, level, console),
        'loggers': {
            f'{name}': {
                'level': level,
                'handlers': handlers,
            },
        }
    })
    return logging.getLogger(name)

caseLogger = setup_logger("case", False)
debugLogger = setup_logger("debug", False)

In [33]:
class LoggerWrapper:
    def __init__(self, logger, logger_props):
        self._logger_props = logger_props
        self._logger = logger

    def __getattr__(self, attr):
        local_props = self._logger_props
        def wrapped(message):
            return getattr(self._logger, attr)(message, extra=local_props)
        return wrapped

In [34]:
debugLogger.info("test debug logger", extra={'component':"Comp1"})
caseLogger.info("test case logger", extra={'case':"Case1"})

In [35]:
class CaseCapsule:
    def __init__(self, id: str, offset=0, logger: logging.Logger=caseLogger):
        # TODO
        self._id = id
        self._last_seen = datetime.datetime.today()
        self._counter = -1
        self._log = logger
        self.add_seconds(offset)

    @property
    def id(self):
        return self._id

    @property
    def last_seen(self):
        return self._last_seen

    @property
    def counter(self):
        return self._counter

    def add_seconds(self, seconds):
        self._counter += 1
        timedelta = datetime.timedelta(seconds=seconds)
        self._last_seen += timedelta

    @property
    def log(self):
        return LoggerWrapper(self._log, {'case':str(self)})

    def __str__(self):
        return f"id {self._id}::{self._last_seen}::count {self._counter}"

In [36]:
case = CaseCapsule("42")
print(case)
case.add_seconds(100)
print(case)

id 42::2021-01-12 17:17:59.639498::count 0
id 42::2021-01-12 17:19:39.639498::count 1


In [37]:
newLogger = LoggerWrapper(caseLogger, {'case': 'hello_world'})

newLogger.debug('test log')

In [38]:
class Component:
    def __init__(self, name, pre_sub_function, post_sub_function, debug_logger=debugLogger, sub_component=None):
        self._name = name
        self._pre_sub_function = pre_sub_function
        self._post_sub_function = post_sub_function
        self._sub_component = sub_component
        self._debug_logger = LoggerWrapper(debug_logger, {'component':name})

    def exec(self, case: CaseCapsule):
        if self._pre_sub_function:
            self._debug_logger.debug('before calling pre sub functions')
            self._pre_sub_function(case, self._debug_logger, self._name)
            self._debug_logger.debug('after calling pre sub functions')
        
        if self._sub_component:
            self._debug_logger.debug('before calling sub component')
            self._sub_component.exec(case)
            self._debug_logger.debug('after calling sub component')
        
        if self._post_sub_function:
            self._debug_logger.debug('before calling post sub functions')
            self._post_sub_function(case, self._debug_logger, self._name)
            self._debug_logger.debug('after calling post sub functions')

In [39]:
def test_pre_fun(case:CaseCapsule, logger: logging.Logger, name):
    logger.debug('some demo pre function')
    case.add_seconds(2)
    case.log.debug(f'executed pre of component {name}')

def test_post_fun(case:CaseCapsule, logger: logging.Logger, name):
    logger.debug('some demo post function')
    case.add_seconds(2)
    case.log.debug(f'executed post of component {name}')

comp = Component("TestComp", test_pre_fun, test_post_fun)

case = CaseCapsule("1")

comp.exec(case)

In [40]:
def simulate_timeout(ceil=10, likelihood=0.9):
    # decide if timeout should be added
    is_timeout = random.random()

    if is_timeout > likelihood:
        # add timeout between range of ceil
        return random.randrange(ceil*10)/10
    
    return 0.1

print(case)
case.add_seconds(simulate_timeout(10, 0.6))
print(case)

id 1::2021-01-12 17:18:03.775010::count 2
id 1::2021-01-12 17:18:05.875010::count 3


In [41]:
def create_simulation_data(component, nr_cases, logger):
    for i in range(nr_cases):
        case = CaseCapsule(f"{i}", random.randrange(1000)/100, logger)
        component.exec(case)


# Define simulation scenarios



## Nice weather case simulation
Nice weather case scenario models the process optimal response time and throughput

In [42]:
def nice_weather_pre_function(case:CaseCapsule, logger: logging.Logger, name):
    logger.debug('executing pre function')
    case.add_seconds(simulate_timeout(2, 0.9))
    case.log.debug(f'executed pre function of component {name}')

def nice_weather_post_function(case:CaseCapsule, logger: logging.Logger, name):
    logger.debug('executing post function')
    case.add_seconds(simulate_timeout(2, 0.9))
    case.log.debug(f'executed post function of component {name}')

notification_service = Component("notification_service", nice_weather_pre_function, nice_weather_post_function, debugLogger)
correlator = Component("correlator", nice_weather_pre_function, nice_weather_post_function, debugLogger, notification_service)
analytics = Component("analytics", nice_weather_pre_function, nice_weather_post_function, debugLogger, correlator)
workflow_engine = Component("workflow_engine", nice_weather_pre_function, nice_weather_post_function, debugLogger, analytics)
worker = Component("worker", nice_weather_pre_function, nice_weather_post_function, debugLogger, workflow_engine)
indicator_parser = Component("indicator_parser", nice_weather_pre_function, nice_weather_post_function, debugLogger, worker)
email_service = Component("email_service", nice_weather_pre_function, nice_weather_post_function, debugLogger, indicator_parser)
guardia_api = Component("guardia_api", nice_weather_pre_function, nice_weather_post_function, debugLogger, email_service)
auth_proxy = Component("auth_proxy", nice_weather_pre_function, None, debugLogger, guardia_api)

In [43]:
caseLogger = setup_logger("nice_weather_case")
case = CaseCapsule("1", 0, caseLogger)

auth_proxy.exec(case)

CASE::id 1::2021-01-12 17:18:00.021283::count 1::DEBUG::executed pre function of component auth_proxy
CASE::id 1::2021-01-12 17:18:00.121283::count 2::DEBUG::executed pre function of component guardia_api
CASE::id 1::2021-01-12 17:18:00.221283::count 3::DEBUG::executed pre function of component email_service
CASE::id 1::2021-01-12 17:18:00.321283::count 4::DEBUG::executed pre function of component indicator_parser
CASE::id 1::2021-01-12 17:18:00.421283::count 5::DEBUG::executed pre function of component worker
CASE::id 1::2021-01-12 17:18:00.521283::count 6::DEBUG::executed pre function of component workflow_engine
CASE::id 1::2021-01-12 17:18:00.621283::count 7::DEBUG::executed pre function of component analytics
CASE::id 1::2021-01-12 17:18:01.321283::count 8::DEBUG::executed pre function of component correlator
CASE::id 1::2021-01-12 17:18:01.421283::count 9::DEBUG::executed pre function of component notification_service
CASE::id 1::2021-01-12 17:18:01.521283::count 10::DEBUG::execu

## Simulating inconsistent and slow correlator

In [44]:
def long_pre_function(case:CaseCapsule, logger: logging.Logger, name):
    logger.debug('executing pre function')
    case.add_seconds(simulate_timeout(10, 0.4))
    case.log.debug(f'executed pre function of component {name}')

def long_post_function(case:CaseCapsule, logger: logging.Logger, name):
    logger.debug('executing post function')
    case.add_seconds(simulate_timeout(5, 0.3))
    case.log.debug(f'executed post function of component {name}')

notification_service = Component("notification_service", nice_weather_pre_function, nice_weather_post_function, debugLogger)
correlator = Component("correlator", long_pre_function, long_post_function, debugLogger, notification_service)
analytics = Component("analytics", nice_weather_pre_function, nice_weather_post_function, debugLogger, correlator)
workflow_engine = Component("workflow_engine", nice_weather_pre_function, nice_weather_post_function, debugLogger, analytics)
worker = Component("worker", nice_weather_pre_function, nice_weather_post_function, debugLogger, workflow_engine)
indicator_parser = Component("indicator_parser", nice_weather_pre_function, nice_weather_post_function, debugLogger, worker)
email_service = Component("email_service", nice_weather_pre_function, nice_weather_post_function, debugLogger, indicator_parser)
guardia_api = Component("guardia_api", nice_weather_pre_function, nice_weather_post_function, debugLogger, email_service)
auth_proxy = Component("auth_proxy", nice_weather_pre_function, None, debugLogger, guardia_api)

In [45]:
caseLogger = setup_logger("long_correlator_case", False)

create_simulation_data(auth_proxy, 1000, caseLogger)

In [46]:
notification_service = Component("notification_service", nice_weather_pre_function, nice_weather_post_function, debugLogger)
correlator = Component("correlator", long_pre_function, long_post_function, debugLogger, notification_service)
analytics = Component("analytics", nice_weather_pre_function, nice_weather_post_function, debugLogger, correlator)
workflow_engine = Component("workflow_engine", nice_weather_pre_function, nice_weather_post_function, debugLogger, analytics)
worker = Component("worker", long_pre_function, nice_weather_post_function, debugLogger, workflow_engine)
indicator_parser = Component("indicator_parser", nice_weather_pre_function, nice_weather_post_function, debugLogger, worker)
email_service = Component("email_service", nice_weather_pre_function, nice_weather_post_function, debugLogger, indicator_parser)
guardia_api = Component("guardia_api", nice_weather_pre_function, nice_weather_post_function, debugLogger, email_service)
auth_proxy = Component("auth_proxy", nice_weather_pre_function, None, debugLogger, guardia_api)

In [47]:
caseLogger = setup_logger("long_correlator_worker_case", False)

create_simulation_data(auth_proxy, 1000, caseLogger)